In [ ]:
dataset_root = '/kaggle/input/bharatnatyam-mudra-dataset-entire'
print("Contents:")
for subfolder in os.listdir(dataset_root):
    print(subfolder)


In [ ]:
dataset_root = '/kaggle/input/bharatnatyam-mudra-dataset-entire/Bharatanatyam-Mudra-Dataset-master'
print("Subfolders:")
for f in os.listdir(dataset_root):
    print(f)


In [ ]:
import os

one_hand_path = os.path.join(dataset_root, 'ONE-HAND MUDRAS')
one_hand_classes = [d for d in os.listdir(one_hand_path) if os.path.isdir(os.path.join(one_hand_path, d))]
print(f"ONE-HAND MUDRAS classes ({len(one_hand_classes)}):")
for cls in one_hand_classes:
    print(cls)

print("\nImage count per class (ONE-HAND MUDRAS):")
for cls in one_hand_classes:
    cls_folder = os.path.join(one_hand_path, cls)
    img_files = [f for f in os.listdir(cls_folder) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
    print(f"{cls}: {len(img_files)}")


In [ ]:
import os
import shutil

root = "/kaggle/input/bharatnatyam-mudra-dataset-entire/Bharatanatyam-Mudra-Dataset-master"

one_hand_dir = os.path.join(root, "ONE-HAND MUDRAS")
right_dir = os.path.join(root, "Right Mudras")
wrong_dir = os.path.join(root, "Wrong Mudras")
wrong_onehand_dir = os.path.join(root, "Wrong One-Hand Mudras")

clean_root = "/kaggle/working/clean_mudra_dataset"
os.makedirs(clean_root, exist_ok=True)

# Helper to copy images
def copy_all(src, dest):
    os.makedirs(dest, exist_ok=True)
    for f in os.listdir(src):
        if f.lower().endswith((".jpg", ".jpeg", ".png")):
            shutil.copy(os.path.join(src, f), dest)


print("🚀 Building unified clean mudra dataset...\n")

# 1️⃣ Copy TRUE mudras
for cls in os.listdir(one_hand_dir):
    src = os.path.join(one_hand_dir, cls)
    if os.path.isdir(src):
        dest = os.path.join(clean_root, cls)
        copy_all(src, dest)
        print("✔ Copied:", cls)

# 2️⃣ Copy WRONG mudras → name them with _WRONG
def copy_wrong(src_dir):
    for cls in os.listdir(src_dir):
        src = os.path.join(src_dir, cls)
        if os.path.isdir(src):
            wrong_name = cls + "_WRONG"
            dest = os.path.join(clean_root, wrong_name)
            copy_all(src, dest)
            print("❌ Copied wrong:", wrong_name)

copy_wrong(wrong_onehand_dir)
copy_wrong(wrong_dir)
copy_wrong(right_dir)

print("\n🎉 Clean dataset created at:", clean_root)
print("Classes found:", os.listdir(clean_root))


Training and saving the model

In [ ]:
import os
import json
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG19
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

# NEW CLEAN DATASET
data_dir = "/kaggle/working/clean_mudra_dataset"
model_name = "final_vgg19_mudra_classifier"

# Data generator
datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,
    rotation_range=20,
    zoom_range=0.15,
    shear_range=0.15,
    width_shift_range=0.10,
    height_shift_range=0.10,
    horizontal_flip=True
)

train_data = datagen.flow_from_directory(
    data_dir,
    target_size=(224, 224),
    batch_size=16,
    class_mode='categorical',
    subset='training',
    shuffle=True
)

val_data = datagen.flow_from_directory(
    data_dir,
    target_size=(224, 224),
    batch_size=16,
    class_mode='categorical',
    subset='validation',
    shuffle=False
)

# Model
base_model = VGG19(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

for layer in base_model.layers:
    layer.trainable = False

x = GlobalAveragePooling2D()(base_model.output)
x = Dense(256, activation='relu')(x)
x = Dropout(0.4)(x)
output = Dense(train_data.num_classes, activation='softmax')(x)

model = Model(base_model.input, output)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

callbacks = [
    EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True),
    ReduceLROnPlateau(monitor='val_loss', factor=0.3, patience=2)
]

history = model.fit(
    train_data,
    validation_data=val_data,
    epochs=15,
    callbacks=callbacks,
    verbose=1
)

# Save model + labels
model.save(f"/kaggle/working/{model_name}.h5")

with open(f"/kaggle/working/{model_name}_labels.json", "w") as f:
    json.dump(train_data.class_indices, f)

print("\n🎉 Model saved!")
print("Classes:", train_data.class_indices)

model.summary()


In [ ]:
# -------------------------------------------------------
# 🔥 Fine-tuning: Unfreeze deeper VGG19 layers
# -------------------------------------------------------
for layer in base_model.layers:
    if "block5" in layer.name or "block4" in layer.name:
        layer.trainable = True
    else:
        layer.trainable = False

print("🔓 Unfrozen layers:")
for layer in model.layers:
    if layer.trainable:
        print("Trainable:", layer.name)

# -------------------------------------------------------
# 🔁 Recompile with a MUCH lower learning rate
# -------------------------------------------------------
model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-5),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# -------------------------------------------------------
# 🚀 Train again (fine-tuning)
# -------------------------------------------------------
fine_tune_history = model.fit(
    train_data,
    validation_data=val_data,
    epochs=12,
    callbacks=[
        EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True),
        ReduceLROnPlateau(monitor='val_loss', patience=2, factor=0.2)
    ],
    verbose=1
)

# -------------------------------------------------------
# 💾 Save fine-tuned model
# -------------------------------------------------------
model.save("/kaggle/working/fine_tuned_vgg19_mudra_classifier.keras")
print("🎉 Fine-tuned model saved!")


In [ ]:
import tensorflow as tf
import numpy as np
import json
import requests
from io import BytesIO
from PIL import Image

# ---------------------------------------------------------
# 1️⃣ Load Fine-Tuned Model + Labels
# ---------------------------------------------------------
model_path = "/kaggle/working/fine_tuned_vgg19_mudra_classifier.keras"  # UPDATE if needed
labels_path = "/kaggle/working/final_vgg19_mudra_classifier_labels.json"

model = tf.keras.models.load_model(model_path, compile=False)

with open(labels_path, "r") as f:
    class_map = json.load(f)

# Reverse mapping {0: "MudraName", 1: "MudraName_WRONG"...}
idx_to_class = {v: k for k, v in class_map.items()}

print("✅ Fine-tuned model & label map loaded!")


# ---------------------------------------------------------
# 2️⃣ Download + Preprocess Image From URL
# ---------------------------------------------------------
def load_image_from_url(url):
    try:
        resp = requests.get(url)
        resp.raise_for_status()
    except Exception as e:
        raise ValueError(f"❌ Failed to download image: {e}")

    img = Image.open(BytesIO(resp.content)).convert("RGB")
    img = img.resize((224, 224))

    img = np.array(img).astype("float32") / 255.0
    img = np.expand_dims(img, axis=0)

    return img


# ---------------------------------------------------------
# 3️⃣ Predict Mudra
# ---------------------------------------------------------
def predict_mudra(url):
    img = load_image_from_url(url)

    preds = model.predict(img)
    class_id = np.argmax(preds)
    confidence = preds[0][class_id]

    label = idx_to_class[class_id]

    print("\n🎯 **Prediction Result**")
    print(f"Mudra: {label}")
    print(f"Confidence: {confidence * 100:.2f}%")

    return label, confidence


# ---------------------------------------------------------
# 4️⃣ Test Run
# ---------------------------------------------------------
if __name__ == "__main__":
    # 🔥 Replace with any mudra image URL
    image_url = "https://www.shutterstock.com/image-photo/woman-hand-showing-hamsasyo-hasta-260nw-25102501.jpg"

    predict_mudra(image_url)


In [ ]:
import tensorflow as tf
import numpy as np
import json
import os
import random
from PIL import Image

# ---------------------------------------------------------
# 1️⃣ Load fine-tuned model + labels
# ---------------------------------------------------------
model_path = "/kaggle/working/fine_tuned_vgg19_mudra_classifier.keras"  # update if needed
labels_path = "/kaggle/working/final_vgg19_mudra_classifier_labels.json"
dataset_path = "/kaggle/working/clean_mudra_dataset"  # your cleaned dataset

model = tf.keras.models.load_model(model_path, compile=False)

with open(labels_path, "r") as f:
    class_map = json.load(f)

# Reverse mapping: index → class name
idx_to_class = {v: k for k, v in class_map.items()}

print("✅ Model + labels loaded successfully!")


# ---------------------------------------------------------
# 2️⃣ Preprocess function
# ---------------------------------------------------------
def preprocess(path):
    img = Image.open(path).convert("RGB")
    img = img.resize((224, 224))
    img = np.array(img) / 255.0
    img = np.expand_dims(img, axis=0)
    return img


# ---------------------------------------------------------
# 3️⃣ Pick a random image from your dataset
# ---------------------------------------------------------
def get_random_image(dataset_root):
    # Choose random class folder
    class_folder = random.choice(os.listdir(dataset_root))
    folder_path = os.path.join(dataset_root, class_folder)

    # Pick random image inside it
    image_name = random.choice([
        f for f in os.listdir(folder_path)
        if f.lower().endswith((".jpg", ".jpeg", ".png"))
    ])
    
    image_path = os.path.join(folder_path, image_name)
    return image_path, class_folder


# ---------------------------------------------------------
# 4️⃣ Predict using the model
# ---------------------------------------------------------
def predict_random_image():
    image_path, true_class = get_random_image(dataset_path)
    
    img = preprocess(image_path)
    preds = model.predict(img)

    class_id = np.argmax(preds)
    confidence = preds[0][class_id]
    predicted_label = idx_to_class[class_id]

    print("\n🖼️ Random Image:", image_path)
    print(f"📌 True Class: {true_class}")
    print(f"🎯 Predicted: {predicted_label}")
    print(f"🔥 Confidence: {confidence*100:.2f}%")

    # Show the image
    img_display = Image.open(image_path)
    display(img_display)

    return predicted_label, confidence


# ---------------------------------------------------------
# 5️⃣ Run test
# ---------------------------------------------------------
predict_random_image()

In [ ]:
print(train_data.class_indices)

URL Prediction 

In [ ]:
import tensorflow as tf
import numpy as np
import json
import requests
from io import BytesIO
from PIL import Image

# ---------------------------------------------------------
# 1️⃣ Load fine-tuned model + labels
# ---------------------------------------------------------
model_path = "/kaggle/working/fine_tuned_vgg19_mudra_classifier.keras"
labels_path = "/kaggle/working/final_vgg19_mudra_classifier_labels.json"

model = tf.keras.models.load_model(model_path, compile=False)

with open(labels_path, "r") as f:
    class_map = json.load(f)

# Fix for JSON with string keys
idx_to_class = {int(v): k for k, v in class_map.items()}

print("✅ Model + labels loaded successfully!")


# ---------------------------------------------------------
# 2️⃣ Preprocess function for URL image
# ---------------------------------------------------------
def preprocess_url_image(url):
    response = requests.get(url)
    img = Image.open(BytesIO(response.content)).convert("RGB")
    img = img.resize((224, 224))
    img = np.array(img) / 255.0
    img = np.expand_dims(img, axis=0)
    return img, img.shape


# ---------------------------------------------------------
# 3️⃣ Predict from URL
# ---------------------------------------------------------
def predict_from_url(image_url):
    try:
        img, _ = preprocess_url_image(image_url)
        preds = model.predict(img)

        class_id = np.argmax(preds)
        confidence = preds[0][class_id]
        predicted_label = idx_to_class[class_id]

        print("\n🔗 Image URL:", image_url)
        print(f"🧿 Predicted Mudra: {predicted_label}")
        print(f"🔥 Confidence: {confidence*100:.2f}%")

        # Show the image
        display(Image.open(BytesIO(requests.get(image_url).content)))

        return predicted_label, confidence

    except Exception as e:
        print("❌ Error processing the image:", e)


# ---------------------------------------------------------
# 4️⃣ Test it!
# ---------------------------------------------------------
test_url = "https://sulcdn.azureedge.net/content/images/blogs/24043018-7-arala.jpg"   # 🔥 put your image URL here
predict_from_url(test_url)
